In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install required packages
!pip install -Uqq tf-nightly

# Import necessary libraries
import tensorflow as tf
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from PIL import Image


# Define the input dimensions for your YOLOv5 model
# Assumptions 
model_input_width = 320  
model_input_height = 320  


# Load YOLOv5 model
model_path = '/kaggle/input/yolo-v5/tflite/tflite-tflite-model/1/1.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Load dataset
dataset_path = '/kaggle/input/dataset'
train_csv_path = Path(dataset_path) / 'train.csv'
test_csv_path = Path(dataset_path) / 'test.csv'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)


# Create a dictionary to map object labels to numerical identifiers
label_mapping = {'pedestrian': 1, 'people': 2, 'bicycle': 3, 'car': 4, 'van': 5, 'truck': 6, 'tricycle': 7, 'awning-tricycle': 8, 'bus': 9, 'motor': 10, 'others': 11}


# Preprocess the dataset
def preprocess_image(image_path,model_input_width, model_input_height):
    # Implement preprocessing steps as needed
    img = Image.open(image_path)
    img = img.resize((model_input_width, model_input_height))  #resize
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array

def process_output(output_data):
    # Implement processing of the YOLOv5 output data
    # Extracting bounding box coordinates, confidence scores, and class predictions
    bounding_boxes = output_data[:, :, :4]
    confidence_scores = output_data[:, :, 4]
    class_predictions = output_data[:, :, 5:]

    return bounding_boxes, confidence_scores, class_predictions


# collect result after Running inference on the dataset
results = []

for index, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Running Inference"):
    image_path = Path(dataset_path) / 'images' / f"{row['id']}.jpg"  # Update the path to 'images'
    
    # Check if the image file exists
    if not image_path.is_file():
        print(f"Image not found: {image_path}")
        continue
    
    image_data = preprocess_image(image_path, model_input_width, model_input_height)

    # Initialize YOLO annotations list
    annotations = row['annotations'].split('|')
    
    yolo_annotations = []

    for annotation in annotations:
        bbox_info = annotation.split(',')
        print("bbox_info:", bbox_info)
    
    # Make sure there are at least 5 elements in bbox_info
        if len(bbox_info) >= 5:
            bbox_left_str, bbox_top_str, bbox_width_str, bbox_height_str, object_label_str = bbox_info[:5]

    # Convert string variables to integers
            bbox_left, bbox_top, bbox_width, bbox_height = map(int, [bbox_left_str, bbox_top_str, bbox_width_str, bbox_height_str])
    
    # Map the object label string to a numerical identifier
            object_label = label_mapping.get(object_label_str, -1)
            
            if object_label != -1:
                center_x = (bbox_left + bbox_width / 2) / model_input_width
                center_y = (bbox_top + bbox_height / 2) / model_input_height
                normalized_width = bbox_width / model_input_width
                normalized_height = bbox_height / model_input_height

                yolo_annotations.append(f"{object_label} {center_x} {center_y} {normalized_width} {normalized_height}")
            else:
                print(f"Invalid label: {object_label_str}")
            
    # Combine YOLO annotations with line break
    yolo_annotations_str = "\n".join(yolo_annotations)
    # Define the input and output tensor indices
     # Define the input and output tensor indices
    input_tensor_index = interpreter.get_input_details()[0]['index']
    output_tensor_index = interpreter.get_output_details()[0]['index']

    # Perform inference
    interpreter.set_tensor(input_tensor_index, image_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_tensor_index)

    # Process the output data as needed
    # Save results for evaluation
    results.append({
        'image_filename': row['id'],
        'predicted_boxes': process_output(output_data),
        
    })


# Save the trained model
model.save('/kaggle/working/yolov5_model')

# Save the model weights
model.save_weights('/kaggle/working/yolov5_model_weights.h5')

In [ ]:
# Load test dataset
test_df = pd.read_csv(test_csv_path)

# Create an empty list to store the results for submission
submission_results = []

# Run inference on the test dataset
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running Inference on Test Set"):
    image_path = Path(dataset_path) / 'images' / f"{row['id']}.jpg"

    # Check if the image file exists
    if not image_path.is_file():
        print(f"Image not found: {image_path}")
        continue

    # Preprocess the image
    image_data = preprocess_image(image_path, model_input_width, model_input_height)

    # Perform inference
    interpreter.set_tensor(input_tensor_index, image_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_tensor_index)

    # Process the output data
    predicted_boxes, confidence_scores, class_predictions = process_output(output_data)

    # Convert these predictions to the required annotation format
    annotations = []

    for i in range(len(predicted_boxes)):
        bbox = predicted_boxes[i]
        confidence = confidence_scores[i]
        class_prediction = class_predictions[i]

        # Convert bbox coordinates to integers
        bbox = bbox.astype(int)

        # Get the predicted object label (assuming class_predictions is a one-hot encoded vector)
        object_label = np.argmax(class_prediction) + 1  # +1 because class indices start from 1 in your label_mapping

        # Format the annotation string
        annotation_str = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]},{confidence},{object_label}"

        annotations.append(annotation_str)

    # Combine multiple object annotations with the pipe symbol
    annotations_str = "|".join(annotations)

    # Append results to the submission list
    submission_results.append({'id': row['id'], 'annotations': f"[{annotations_str}]|{object_label}"})

# Create a DataFrame for submission
submission_df = pd.DataFrame(submission_results)

# Save the submission file
submission_df.to_csv('submission3.csv', index=False)
